In [3]:
import numpy as np
import bilby
import json
import glob
import pandas as pd
from tqdm import trange

max_f = 2048
min_f = 23

In [4]:
def get_bns_prior(chirp_mass):
    prior_template = bilby.gw.prior.BNSPriorDict(aligned_spin=True)
    prior_template['chirp_mass'] = bilby.core.prior.Uniform(name = 'chirp_mass', minimum = chirp_mass - 0.01, maximum= chirp_mass + 0.01)
    prior_template['mass_ratio'] = bilby.core.prior.Uniform(name = 'mass_ratio', minimum = 0.125, maximum = 1)
    for ii in [1,2]:
        del prior_template[f"mass_{ii}"]
        prior_template[f"chi_{ii}"].a_prior.maximum = 0.05
        prior_template[f"lambda_{ii}"].maximum = 15000
    prior_template['luminosity_distance'] = bilby.gw.prior.UniformSourceFrame(minimum = 1, maximum = 1500, name = 'luminosity_distance', latex_label='$d_L$')
    
    new_prior = {key: prior_template[key] for key in prior_template.keys()}
    return new_prior

In [5]:
def get_duration(sample, fmin):    
    duration = bilby.gw.utils.calculate_time_to_merger(fmin, sample['mass_1'], 
                sample['mass_2'])
    duration = max(4, 2**(np.ceil(np.log2(duration))))
    return duration

In [6]:
def generate_ini(params, name, fmax, fmin, subdir):
    with open('template.ini', 'r') as rr:
        template = rr.readlines()

    params['chirp_mass'] = bilby.gw.conversion.component_masses_to_chirp_mass(mass_1=params['mass_1'], mass_2 = params['mass_2'])
    
    duration = get_duration(params, min_f)
  
    prior_here = get_bns_prior(params['chirp_mass'])
    #prior_here['luminosity_distance'].cosmology = None
    #prior_here['luminosity_distance'].unit = None
    for jj, line in enumerate(template):
        if line.startswith("duration="):
            template[jj] = f"duration={duration}\n"
        if line.startswith("sampling-frequency="):
            template[jj] = f"sampling-frequency={fmax * 2}\n"
        if line.startswith("maximum-frequency="):
            template[jj] = f"maximum-frequency={fmax * 7/8}\n"
        if line.startswith("injection-dict="):
            template[jj] = f"injection_dict={params}\n"
        if line.startswith("label="):
            template[jj] = f"label={name}\n"
        if line.startswith("prior-dict="):
            template[jj] = "prior-dict={"
            for key in prior_here.keys():
                template[jj] += f"{key}:{prior_here[key]},"
            template[jj] = template[jj].strip(',')
            template[jj] += "}\n"
        if line.startswith("outdir="):
            template[jj] = f"outdir={subdir}/outdir_{name}\n"
    return template

In [7]:
def get_and_write_ini(indx, df, fmax, fmin, subdir):
    params_here = {key: df.loc[key][0][indx] for key in df.index if "Unnamed" not in key}
    print(params_here)
    name_here = f"injection_{indx}"
    ini = generate_ini(params_here, name_here, fmax, fmin, subdir)
    with open(f"{subdir}/{name_here}.ini", "w") as ww:
        ww.writelines(ini)

In [18]:
events = pd.read_json("ABHT_QMCRMF2/found_injections-abht_qmcrmf2_0.json")
params_here = {key: events.loc[key][0][1] for key in events.index if "Unnamed" not in key}
params_here

{'SNR_passed': True,
 'all_cuts_passed': True,
 'chi_1': -5.853780158931462e-07,
 'chi_2': 0.005960292522309001,
 'chirp_mass': 1.496341936380329,
 'comoving_distance': 104.02272610468582,
 'dec': -0.296104124056984,
 'geocent_time': 0.0,
 'lambda_1': 39.79150900918767,
 'lambda_2': 197.48241220808288,
 'luminosity_distance': 106.48126706100513,
 'mass_1': 1.896379422096461,
 'mass_1_source': 1.8525939982288722,
 'mass_2': 1.560882759862513,
 'mass_2_source': 1.524843604168323,
 'mass_ratio_passed': True,
 'optimal_snr_squared': 1181.210280110161,
 'phase': 1.518294035534122,
 'psi': 2.820783497402834,
 'ra': 2.198865209353751,
 'redshift': 0.023634657085928,
 'theta_jn': 1.9591833720842131}

In [16]:
events.loc["mass_1"]

injections    [1.691771974207073, 1.896379422096461, 1.99601...
Name: mass_1, dtype: object

In [29]:
events.iloc[0]["injections"]

[True, True, True, True, True, True, True, True, True, True, True, True, True]

In [30]:
for i in trange(len(events.iloc[0]["injections"])):
    get_and_write_ini(i, events, max_f, min_f, subdir="ABHT_QMCRMF2")

  0%|          | 0/13 [00:00<?, ?it/s]14:37 bilby INFO    : No prior given, using default BNS priors in /cvmfs/software.igwn.org/conda/envs/igwn-testing/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_bns_tides_on.prior.


{'SNR_passed': True, 'all_cuts_passed': True, 'chi_1': 0.036850625561290005, 'chi_2': -0.0017134324965930002, 'chirp_mass': 1.34858038388442, 'comoving_distance': 273.2978119858125, 'dec': -1.01631389956222, 'geocent_time': 0.0, 'lambda_1': 145.15471676424988, 'lambda_2': 460.8362669469534, 'luminosity_distance': 290.426592946818, 'mass_1': 1.691771974207073, 'mass_1_source': 1.5919946348684362, 'mass_2': 1.420644725248986, 'mass_2_source': 1.336857930697522, 'mass_ratio_passed': True, 'optimal_snr_squared': 206.30922514467784, 'phase': 6.254272698020112, 'psi': 1.955631662267679, 'ra': 3.9933340827033863, 'redshift': 0.06267441934368201, 'theta_jn': 1.053762046933667}


  8%|▊         | 1/13 [00:00<00:03,  3.23it/s]14:37 bilby INFO    : No prior given, using default BNS priors in /cvmfs/software.igwn.org/conda/envs/igwn-testing/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_bns_tides_on.prior.


{'SNR_passed': True, 'all_cuts_passed': True, 'chi_1': -5.853780158931462e-07, 'chi_2': 0.005960292522309001, 'chirp_mass': 1.496341936380329, 'comoving_distance': 104.02272610468582, 'dec': -0.296104124056984, 'geocent_time': 0.0, 'lambda_1': 39.79150900918767, 'lambda_2': 197.48241220808288, 'luminosity_distance': 106.48126706100513, 'mass_1': 1.896379422096461, 'mass_1_source': 1.8525939982288722, 'mass_2': 1.560882759862513, 'mass_2_source': 1.524843604168323, 'mass_ratio_passed': True, 'optimal_snr_squared': 1181.210280110161, 'phase': 1.518294035534122, 'psi': 2.820783497402834, 'ra': 2.198865209353751, 'redshift': 0.023634657085928, 'theta_jn': 1.9591833720842131}


 15%|█▌        | 2/13 [00:00<00:03,  3.00it/s]14:37 bilby INFO    : No prior given, using default BNS priors in /cvmfs/software.igwn.org/conda/envs/igwn-testing/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_bns_tides_on.prior.


...

 85%|████████▍ | 11/13 [00:03<00:00,  3.04it/s]14:37 bilby INFO    : No prior given, using default BNS priors in /cvmfs/software.igwn.org/conda/envs/igwn-testing/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_bns_tides_on.prior.


{'SNR_passed': True, 'all_cuts_passed': True, 'chi_1': 0.0025824486741290002, 'chi_2': 0.00021109714857800002, 'chirp_mass': 1.215075253064641, 'comoving_distance': 208.91658738981067, 'dec': 0.80122188964582, 'geocent_time': 0.0, 'lambda_1': 163.26307726354125, 'lambda_2': 1167.4785444521467, 'luminosity_distance': 218.89006186281725, 'mass_1': 1.639389951435409, 'mass_1_source': 1.564693030997879, 'mass_2': 1.194279850166262, 'mass_2_source': 1.139863860322049, 'mass_ratio_passed': True, 'optimal_snr_squared': 394.0840139172895, 'phase': 3.035217553537722, 'psi': 3.041094766347139, 'ra': 1.157861605860497, 'redshift': 0.047739025455933004, 'theta_jn': 0.8864561100495291}


 92%|█████████▏| 12/13 [00:03<00:00,  2.94it/s]14:37 bilby INFO    : No prior given, using default BNS priors in /cvmfs/software.igwn.org/conda/envs/igwn-testing/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_bns_tides_on.prior.


{'SNR_passed': True, 'all_cuts_passed': True, 'chi_1': 0.00682476252103, 'chi_2': 0.012935992245701002, 'chirp_mass': 1.746425526760118, 'comoving_distance': 269.40103390446075, 'dec': 0.09047290243313201, 'geocent_time': 0.0, 'lambda_1': 22.768225040463772, 'lambda_2': 42.93794891110265, 'luminosity_distance': 286.04121195982935, 'mass_1': 2.061865389327998, 'mass_1_source': 1.9419183147556651, 'mass_2': 1.9521659249278551, 'mass_2_source': 1.838600513244406, 'mass_ratio_passed': True, 'optimal_snr_squared': 347.28763638261285, 'phase': 1.024316724783137, 'psi': 1.6577855434504252, 'ra': 2.370240646025308, 'redshift': 0.06176731207534, 'theta_jn': 0.7644853143329261}


100%|██████████| 13/13 [00:04<00:00,  2.90it/s]


In [11]:
events.iloc[events['luminosity_distance'].argmax()]

KeyError: 'luminosity_distance'

In [1]:
pwd


'/home/jacob.golomb/GWs_and_pulsars/may8_aligned/PE'

In [13]:
events

,injections
SNR_passed,"[True, True, True, True, True, True, True, Tru..."
all_cuts_passed,"[True, True, True, True, True, True, True, Tru..."
chi_1,"[0.036850625561290005, -5.853780158931462e-07,..."
chi_2,"[-0.0017134324965930002, 0.005960292522309001,..."
chirp_mass,"[1.34858038388442, 1.496341936380329, 1.618889..."
comoving_distance,"[273.2978119858125, 104.02272610468582, 180.32..."
dec,"[-1.01631389956222, -0.296104124056984, 0.8105..."
geocent_time,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
lambda_1,"[145.15471676424988, 39.79150900918767, 26.970..."
lambda_2,"[460.8362669469534, 197.48241220808288, 102.89..."
